### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

### Pegando os jogos do dia

In [2]:
# Obter a data de hoje
hoje = date.today()
ontem = hoje - timedelta(days=1)

### Atualizando os datasets que tem jogo hoje

In [3]:
dia = ontem

In [4]:
# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)

Jogos do dia 2023-11-20



100%|██████████| 88/88 [01:13<00:00,  1.19it/s]


In [5]:
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Dataset de temporada atualizado com 60 jogos.

----------------------------------------------------------

Atenção! 28 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
56,2023-11-20,Eua Ncaa,21:00,Campbell,N.Carolina Central,NaN,NaN,Não iniciado
57,2023-11-20,Eua Ncaa,21:00,NJIT,Delaware State,NaN,NaN,Não iniciado
58,2023-11-20,Eua Ncaa,21:00,St. Peters,UMass Lowell,NaN,NaN,Não iniciado
59,2023-11-20,Eua Ncaa,21:00,Kentucky,Saint Josephs,NaN,NaN,Não iniciado
60,2023-11-20,Eua Ncaa,21:00,Bethune Cookman,Charleston Southern,NaN,NaN,Não iniciado
61,2023-11-20,Eua Ncaa,21:00,Bucknell,Southern Indiana,NaN,NaN,Não iniciado
62,2023-11-20,Eua Nba,21:00,Washington Wizards,Milwaukee Bucks,110.0,117.0,4º Quarto5'
63,2023-11-20,Eua Nba,21:00,Detroit Pistons,Denver Nuggets,87.0,84.0,4º Quarto3'
67,2023-11-20,Eua Nba,21:00,Charlotte Hornets,Boston Celtics,91.0,100.0,4º Quarto7'
68,2023-11-20,Eua Ncaa,21:00,Troy,Sam Houston St.,NaN,NaN,Não iniciado
